# Iterators

### Protocol of iteration

In [2]:
#if we have
xs = []
for x in xs:
    #body
    pass

# we actually have

it = xs.__iter__()
while True:
    try:
        x = it.__next__()
    except StopIteration:
        break
    #body
    pass

### How can we define iterator?

In [4]:
class Iterator:
    def __next__(self):
        # if self.has_more_elements():
        #     return self.next_element()
        raise StopIteration


it = Iterator()
elem = next(it, 0)

## Iterable

In [5]:
class Iterable:
    def __iter__(self):
        return Iterator()


x = Iterable()

it = iter(x) # calls x.__iter()

Iterator is iterable (good practice)



In [6]:
class Iterator:
    def __next__(self):
        # if self.has_more_elements():
        #     return self.next_element()
        raise StopIteration
    def __iter__(self):
        return self



### Example

In [9]:
class MyRange:
    def __init__(self, start, stop):
        self.start = start
        self.stop = stop
    def __iter__(self):
        return RangeIterator(self.start, self.stop)

class RangeIterator:
    def __init__(self, start, stop):
        self.start = start
        self.stop = stop

    def __iter__(self):
        return self

    def __next__(self):
        if self.start < self.stop:
            res = self.start
            self.start +=1
            return res
        raise StopIteration

In [10]:
r = MyRange(0, 10)
assert not hasattr(r, "__contains__")
assert 8 in r # O(N)

Iterator and iterable are different

In [13]:
r = MyRange(0, 100)
print(sum(r))
print(sum(r))

it = iter(r)
print(sum(it))
print(sum(it)) ## will be 0 since iteration is already over

4950
4950
4950
0


## more compact way to write iterator -- function with closure

In [ ]:
def MyRange(start, stop):
    def step():
        nonlocal start ## state of iterator (closure)
        res = start
        start += 1
        return res
    # iter(step_fn, sentinel)
    return iter(step, stop)

### More convenient way -- Generators

In [17]:
def Myange(start, stop):
    while start < stop:
        yield start
        start += 1

In [18]:
def g():
    print("started")
    x = 42
    yield x # generator paused here, value of x returned
    print("yielded once")
    x += 1
    yield x # generator paused here, value of x returned
    print("yielded twice, done")
    # finished, raised StopIteration

it = g()

for x in it:
    print(x)

started
42
yielded once
43
yielded twice, done


## Applications
all is lazy

In [20]:
def MyUnique(xs):
    seen = set()
    for item in xs:
        if item in seen:
            continue
        seen.add(item)
        yield item

xs = [1, 1, 2, 3, 1]
assert list(MyUnique(xs)) == [1, 2, 3]

In [22]:
def MyMap(func, xs, *rest):
    for args in zip(xs, *rest):
        yield func(*args)

xs = [1, 2, 3, 4]

assert list(MyMap(lambda  x: x * x, xs)) == [1, 4, 9, 16]

In [24]:
def chain(*xss):
    for xs in xss:
        for item in xs:
            yield item

In [25]:
#another way
def chain(*xss):
    for xs in xss:
        yield from xs #special operator
xs = [1, 2, 3]
ys = [92]

assert list(chain(xs, ys)) == [1, 2, 3, 92]

In [26]:
def count(start=0):
    while True: # infinite generator!
        yield start
        start += 1

In [27]:
## iterate through the Binary tree
class BinaryTree:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

    def __iter__(self):
        return self.pre_order

    @property
    def pre_order(self):
        yield self
        if self.left:
            yield from self.left
        if self.right:
            yield from self.right

    @property
    def post_order(self):
        pass

### Generators-statements

In [29]:
map(lambda x: x * x, xs)

In [30]:
(x * x for x in xs)


<generator object <genexpr> at 0x0000024D461DCF48>

In [33]:
sum(x**2 for x in range(10)) # no ()

285

In [36]:
map(lambda s: len(s), ["", "a", "foo"]) # :(


In [37]:
map(len, ["", "a", "foo"]) # ok!

## itertools

In [39]:
from itertools import islice

xs = range(10)

assert list(islice(xs, 2, 8, 3)) == [2, 5]
# islice is lazy

In [40]:
sum(range(0, 10**9, 10**8)) # fast

sum(islice(range(10**9), 0, None, 10**8)) # slow

4500000000

In [41]:

from itertools import islice

def take(n, xs):
    return list(islice(xs, 0, n))

In [43]:
from itertools import count, cycle, repeat, islice

assert take(3 , count(start=1, step=2)) == [1, 3, 5]
assert take(3, cycle(["loves", "doesn't love"])) == ["loves", "doesn't love", "loves"]
assert take(3, repeat(92)) == list(repeat(92, times=3))

In [45]:
from itertools import dropwhile, takewhile

assert list(dropwhile(lambda x: x < 5, range(10))) == [5, 6, 7, 8, 9]

In [67]:
from itertools import chain

assert list(chain(range(2), "abc")) == [0, 1, "a", "b", "c"]

xs = [range(0, i) for i in range(5)]
assert list(chain(*xs)) == list(chain.from_iterable(xs)) == [0, 0, 1, 0, 1, 2, 0, 1, 2, 3]
# chain.from_iterable() is for infinite iterables

In [68]:
from itertools import chain, count, islice

xs = (range(0, i) for i in count())

assert sum(chain.from_iterable(islice(xs, 1000))) == 166167000

In [75]:
it = iter(range(3))

a = b = it
assert list(a) == [0, 1, 2]

assert list(b) == []

In [76]:
from itertools import tee

it = iter(range(3))

a, b, c = tee(it, 3)
assert list(a) == list(b) == list(c) == [0, 1, 2]
# O(N) tee saves list of observed iterators

In [77]:
from itertools import product, combinations, permutations
from itertools import combinations_with_replacement

assert list(product("AB", repeat=2)) == [("A", "A"), ("A", "B"), ("B", "A"), ("B", "B")]

for x in range(3):
    for y in range(5):
        pass


#same as

for x,y in combinations(range(5), 2):
    pass

### Example : graph

In [83]:
from itertools import combinations

def build_graph(words, mismatch_percent):
    for (i, u), (j,v) in combinations(enumerate(words), 2):
        if len(u) != len(v):
            continue



In [84]:
from itertools import groupby

def sorted_runs(xs):
    indices = range(len(xs) - 1)

    def is_increasing(idx):
        return xs[idx] < xs[idx + 1]

    return groupby(indices, is_increasing)

xs = [1, 2, 3, 5, 2, 0, 3, 1]

for is_increasing, group in sorted_runs(xs):
    print("<" if is_increasing else ">", sum(1 for _ in group))  #  magic O(1)

< 3
> 2
< 1
> 1


## Generators and IO

In [1]:
from contextlib import ExitStack, AbstractContextManager  # can open n files
import heapq

def merge_sorted_files(inputs, result):
    with open(result, 'w') as result, ExitStack() as stack:
        files = [stack.enter_context(open(f)) for f in inputs]
        for line in heapq.merge(*files):
            result.write(line)

#merge_sorted_files(["10gb.txt", "20gb.txt"]), "merged.txt")


## Generators - pattern

In [1]:
import os
# How we wrote a context manager
class cd:
    def __init__(self, path):
        self.path = path

    def __enter__(self):
        self.saved_cwd = os.getcwd()
        os.chdir(self.path)

    def __exit__(self, exc_type, exc_val, exc_tb):
        os.chdir(self.saved_cwd)


#Same code

import os
from contextlib import contextmanager

@contextmanager
def cd(path):   # init
    old_path = os.getcwd() #__enter__
    os.chdir()
    try:
        yield    # ______________
    finally:
        os.chdir(old_path)  # __exit__

### Temporary folder

In [2]:
import tempfile
import shutil

@contextmanager
def tempdir(): # init
    outdir = tempfile.mkdtemp() #__enter__
    try:  # how we will catch these exceptions?
        yield outdir #------
    finally:
        shutil.rmtree(outdir) #__exit__

## Co-routines

In [3]:
def sum_and_sumsq(it):
    s = s2 = 0
    for item in it:
        s += item
        s2 += item*item
    return s, s2

In [4]:
# what we want to do?

import itertools

def sum_powers(it, p):
    acc = 0
    for item in it:
        acc += item ** p
    return acc

def sum_and_sumsq(it):
    it1,it2 = itertools.tee(it, 2)
    s = s2 = 0
    return sum_powers(it1, 1), sum_powers(it2, 2) #  we store all elements because we did not iterate throught the second iterator

### Co-Routines

In [8]:
def printer():
    while True:
        item = yield #we can pass elements to iterator
        print(item)

p = printer()

p.send(None) # next(p)
p.send(92)

92


In [13]:
def running_sum():
    acc = 0
    while True:
        acc += yield acc

s = running_sum()
s.send(None) #we should send None first every time

0

In [14]:
s.send(1)

1

In [15]:
s.send(1)



2

In [16]:
s.send(1)


3

In [18]:
# then we will write special decorator which will start coroutine (calls next first time)
import functools

def coroutine(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen

    return inner


In [22]:
@coroutine
def logger():
    with open("output.txt", 'w') as f:
        while True:
            item = yield
            print(item, file=f)

l = logger()
l.send(92)
del l

In [23]:
## exception will be thrown if generator has been deleted
@coroutine
def printer():
    try:
        while True:
            item = yield
            print(item)
    finally:
        print("Cleaning up")

p = printer()
p.send(92)
p.close() #cleaning up



92
Cleaning up


In [25]:
# so we will rewrite our coroutine for sum of powers
@coroutine
def sum_powers_coro(p):
    acc = 0
    while True:
        item = yield #yield is the result of send
        acc += item ** p

In [26]:
#How we will print the result?
STOP = object()

@coroutine
def sum_powers_coro(p):
    acc = 0
    while True:
        item = yield
        if item is STOP:
            return acc # throw exception stopiteration
        acc += item ** p

def result(coro):
    try:
        coro.send(STOP)
    except StopIteration as e:
        return e.value  # acc will be written to e.value

In [28]:
def sum_and_sumsq(it):
    s = sum_powers_coro(p=1)
    s2 = sum_powers_coro(p=2)
    for item in it:
        s.send(item)
        s2.send(item)
    return result(s), result(s2)


assert sum_and_sumsq(iter(range(10))) == (45, 285)

In [30]:
from contextlib import AbstractContextManager
# how it is implemented?

class Manager(AbstractContextManager):
    def __init__(self, co):
        self._co = co
    def __enter__(self):
        return next(self._co)

    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_val is not None:
            try:
                self._co.throw(exc_val)
            except StopIteration:
                pass
        self._co.close()
        return True


def contextmanager(f):
    def inner(*args, **kwargs):
        return Manager(f(*args, **kwargs))

    return inner

* send, throw, close -- can send values or exceptions to generator
* __next__ -- is .send(None)
* generators > iterators

Coroutine is a function which we can call several times and return values several times